## Avances Laboratorio #3 
##### Data Science  -  Universidad del Valle de Guatemala

- Gustavo Andres Gonzalez Pineda 21438
- Maria Marta Ramirez Gil 21342

--------------------------------------------------------------------------------------- 

In [8]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.metrics import mean_squared_error
import math

# Cargar los datos
consumo = pd.read_csv('data/CONSUMO.csv')
importacion = pd.read_csv('data/IMPORTACION.csv')
precios = pd.read_csv('data/PRECIOS.csv')

# Mostrar una vista previa de los datos
print("CONSUMO")
display(consumo)
print()
print ("--------------------------------------------------------")
print("IMPORTACION")
display(importacion)
print()
print ("--------------------------------------------------------")
print("PRECIOS")
display(precios)

CONSUMO


,Fecha,GasLicuado,Regular,Superior,Diesel
0,01/2000,194410.48,202645.20,308156.82,634667.06
1,02/2000,174710.55,205530.96,307766.31,642380.66
2,03/2000,189234.07,229499.56,331910.29,699807.25
3,04/2000,174330.61,210680.40,315648.08,586803.98
4,05/2000,191745.15,208164.34,319667.97,656948.20
...,...,...,...,...,...
288,01/2024,548124.45,830708.13,658083.66,1371766.15
289,02/2024,526897.85,818740.16,654059.60,1352602.93
290,03/2024,523990.91,870771.70,671997.05,1405703.42
291,04/2024,531880.19,847353.15,633520.57,1442103.60



--------------------------------------------------------
IMPORTACION


,Fecha,GasLicuado,Regular,Superior,Diesel
0,01/2001,194065.74,177776.50,373963.96,566101.99
1,02/2001,170703.38,123115.99,243091.07,489525.80
2,03/2001,161837.37,161726.42,312084.38,575559.68
3,04/2001,163048.64,127338.74,285054.89,437745.42
4,05/2001,171518.86,168730.19,300913.67,552609.13
...,...,...,...,...,...
276,01/2024,701570.80,914133.32,712333.33,1415808.13
277,02/2024,916541.70,740662.25,650360.11,1241501.08
278,03/2024,675157.48,838270.93,620077.74,1482045.48
279,04/2024,473940.66,886132.77,687017.96,1294706.12



--------------------------------------------------------
PRECIOS


,Fecha,Superior,Regular,Diesel,GasLicuado
0,01/2021,21.91,21.11,17.61,99.0
1,01/2021,21.91,21.11,17.61,99.0
2,01/2021,21.91,21.11,17.61,99.0
3,01/2021,21.91,21.11,17.61,99.0
4,01/2021,21.91,21.11,17.61,99.0
...,...,...,...,...,...
1304,07/2024,32.79,31.29,28.09,110.0
1305,07/2024,32.63,31.13,27.93,110.0
1306,07/2024,32.63,31.13,27.93,110.0
1307,07/2024,32.63,31.13,27.93,110.0


## Modelo 1 LSTM

In [26]:
# Función para preparar los datos
def prepare_data(series, n_lags):
    series = series.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_series = scaler.fit_transform(series)

    X, y = [], []
    for i in range(n_lags, len(scaled_series)):
        X.append(scaled_series[i - n_lags:i, 0])
        y.append(scaled_series[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler


# Definir el número de lags
n_lags = 10


# Definir el modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = None  # Inicializar como None

    def forward(self, input_seq):
        if self.hidden_cell is None:
            self.hidden_cell = (torch.zeros(1, input_seq.size(0), self.hidden_layer_size),
                                torch.zeros(1, input_seq.size(0), self.hidden_layer_size))
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

    def init_hidden(self, batch_size):
        self.hidden_cell = (torch.zeros(1, batch_size, self.hidden_layer_size),
                            torch.zeros(1, batch_size, self.hidden_layer_size))

### Entrenar Modelo CONSUMO

In [27]:
def train_model(model, train_loader, criterion, optimizer, epochs):
    for i in range(epochs):
        for seq, labels in train_loader:
            model.init_hidden(seq.size(0))  # Inicializar con el tamaño del batch actual
            optimizer.zero_grad()
            y_pred = model(seq)
            single_loss = criterion(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        if i % 25 == 0:
            print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')


# Preparar los datos de entrenamiento y prueba
train_size = int(len(consumo) * 0.8)
test_size = len(consumo) - train_size
consumo_train, consumo_test = consumo[:train_size], consumo[train_size:]

X_train, y_train, scaler = prepare_data(consumo_train['GasLicuado'], n_lags)
X_test, y_test, _ = prepare_data(consumo_test['GasLicuado'], n_lags)

# Convertir a tensores
X_train_tensors = Variable(torch.Tensor(X_train))
y_train_tensors = Variable(torch.Tensor(y_train))
X_test_tensors = Variable(torch.Tensor(X_test))
y_test_tensors = Variable(torch.Tensor(y_test))

# Crear el DataLoader
train_data = torch.utils.data.TensorDataset(X_train_tensors, y_train_tensors)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=False)

# Definir los parámetros de los modelos
params = [
    {'hidden_layer_size': 50, 'epochs': 100, 'batch_size': 32},
    {'hidden_layer_size': 100, 'epochs': 150, 'batch_size': 16}
]

#### Evaluar Modelos

In [28]:
# Entrenar y evaluar los modelos
for i, param in enumerate(params):
    model = LSTMModel(input_size=1, hidden_layer_size=param['hidden_layer_size'], output_size=1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_model(model, train_loader, criterion, optimizer, param['epochs'])

    # Predicciones
    model.eval()
    model.init_hidden(X_train_tensors.size(0))  # Inicializar con el tamaño del batch de entrenamiento
    train_predict = model(X_train_tensors).detach().numpy()
    test_predict = model(X_test_tensors).detach().numpy()

    # Invertir escalado
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calcular RMSE
    train_score = math.sqrt(mean_squared_error(y_train_inv, train_predict))
    test_score = math.sqrt(mean_squared_error(y_test_inv, test_predict))

    print(f'Model {i + 1} - Train Score: {train_score:.2f} RMSE, Test Score: {test_score:.2f} RMSE')

c:\Users\maria\miniconda3\envs\my_env\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   0 loss: 0.63122451
epoch:  25 loss: 0.01205035
epoch:  50 loss: 0.01061409
epoch:  75 loss: 0.01040712
epoch:  99 loss: 0.01040050


RuntimeError: Expected hidden[0] size (1, 49, 50), got [1, 224, 50]